# Práctica 1: Búsquedas no informadas
### [Introducción a los sistemas inteligentes 2019-2](https://fagonzalezo.github.io/iis-2019-2/)

* Juan Sebastian Becerra Barcenas 1118569845
* Juan Sebastian Peña Quintero 1015470303 
* Daniel Fernando Castañeda Torres 1075684238

## 1. El problema de las jarras de agua

Usted tiene 2 jarras, una con capacidad de 3 litros y otra con capacidad de 5 litros. Además, se
cuenta con un grifo. Las jarras se pueden llenar totalmente, vaciar en el piso y vaciar parcialmente el
contenido de una jarra en la otra. Las jarras no tienen ninguna clase de marca. El objetivo es medir
exactamente un litro de agua.

![](https://raw.githubusercontent.com/jsbecerrab/Imagenes-inteligentes/master/water22.png)

### 1.1 
Modele este problema creando una clase:

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

class JarProblem(object):
    

    def __init__(self, initial, goal, capacity): 
        self.capacity = capacity
        self.initial = initial #(0,0)
        self.goal = goal
        
    def actions(self, state):        
        result = []
        if state[0] < self.capacity[0]:
          result.append("Fill A")
        if state[1] < self.capacity[1]:
          result.append("Fill B")
        if state[0] > 0:
          result.append("Dump A")
        if state[1] > 0:
          result.append("Dump B")
        if state[0] > 0 and state[1] < self.capacity[1]:
          result.append("A -> B")
        if state[1] > 0 and state[0] < self.capacity[0]:
          result.append("B -> A")
        return result
      
    def result(self, state, action): 
        newstate = list(state)
        if action == "Fill A":
          newstate[0] = self.capacity[0]
        elif action == "Fill B":
          newstate[1] = self.capacity[1]
        elif action == "Dump A":
          newstate[0] = 0
        elif action == "Dump B":
          newstate[1] = 0
        elif action == "A -> B":
          Bcapacity = self.capacity[1] - state[1]
          if state[0] <= Bcapacity:
            newstate[0] = 0
            newstate[1] = state[1] + state[0]
          else:
            newstate[0] = state[0] - Bcapacity
            newstate[1] = self.capacity[1]
        elif action == "B -> A":
          Acapacity = self.capacity[0] - state[0]
          if state[1] <= Acapacity:
            newstate[1] = 0
            newstate[0] = state[0] + state[1]
          else:
            newstate[1] = state[1] - Acapacity
            newstate[0] = self.capacity[0]
        return tuple(newstate)
    def is_goal(self, state):        
        return self.goal in state
    
    def action_cost(self, s, a, s1): 
        return 1
    
    def h(self, node):               
        return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

### 1.2 
Encuentre una solución utilizando búsqueda en amplitud. Utilice la función `breadth_first_search` en el [notebook](https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) de AIMA. Su código debe imprimir cada uno de los pasos de la solución. 

In [0]:
#Clase Nodo
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
        #print(self.__dict__)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.  

# metodo para crear los hijos de un nodo  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        
# metodo que entrega las acciones usadas para resolver un problema
def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]

# metodo que entrega los estados usados para resolver un problema
def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [0]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [0]:
def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [0]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [6]:
p1_2 = JarProblem((0,0),1,(3,5))
soln1_2 = breadth_first_search(p1_2)
path_actions(soln1_2), path_states(soln1_2)

(['Fill A', 'A -> B', 'Fill A', 'A -> B'],
 [(0, 0), (3, 0), (0, 3), (3, 3), (1, 5)])

### 1.3
Encuentre una solución utilizando búsqueda en profundidad. Utilice la función `depth_first_recursive_search` en el  [notebook](https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) de AIMA. Su código debe imprimir cada uno de los pasos de la solución. 

¿La solución difiere de la solución producida por BFS? Explique.

In [0]:
def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

In [8]:
p1_3 = JarProblem((0,0),1,(3,5))
soln1_3 = depth_first_recursive_search(p1_3)
path_actions(soln1_3), path_states(soln1_3)

(['Fill A',
  'Fill B',
  'Dump A',
  'B -> A',
  'Dump A',
  'B -> A',
  'Fill B',
  'B -> A',
  'Dump A',
  'B -> A'],
 [(0, 0),
  (3, 0),
  (3, 5),
  (0, 5),
  (3, 2),
  (0, 2),
  (2, 0),
  (2, 5),
  (3, 4),
  (0, 4),
  (3, 1)])

###Explicación

Las soluciones difieren en cuanto a que mientras la busqueda en amplitud va probando cada accion disponible que se puede ejecutar en un estado, la busqueda en profundidad unicamente se va centrando en una sola accion hasta llegar al final, por lo que no explora las demas opciones, como se puede evidenciar en los siguientes grafos:
![texto alternativo](https://github.com/jsbecerrab/Imagenes-inteligentes/blob/master/grafos%20de%20estados.png?raw=true)
![texto alternativo](https://github.com/jsbecerrab/Imagenes-inteligentes/blob/master/grafos%20de%20estados2.png?raw=true)

## 2. El problema de los misioneros y los caníbales (MC).

Tres misioneros y tres caníbales deben cruzar un río usando un bote que puede llevar como máximo a dos personas, bajo la restricción de que, para ambos lados del río, si hay misioneros presentes en un lado, no pueden ser superados en número por caníbales (si lo fueran, los caníbales se comerían a los misioneros). El barco no puede cruzar el río por sí mismo sin gente a bordo.

Su objetivo es modelar este problema como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.
![texto alternativo](https://github.com/jsbecerrab/Imagenes-inteligentes/blob/master/cruzacanivales.jpg?raw=true)

### 2.1 Cree una clase para modelar el problema MC
### 2.1 (Cree una clase para modelar el problema MC

Un estado del problema puede representarse de diferentes formas. La sugerencia es usar una tupla de valores indicando el número de misioneros y caníbales en cada lado del rio, así como la posición del bote:

```python
('A', 3, 2, 0, 1)
```

Representa un estado en el cual el bote se encuentra en el lado izquierdo, hay 3 misioneros en el lado izquierdo, 2 caníbales en el lado izquierdo, cero misioneros en el lado derecho y 1 canibal en el lado derecho.

Una acción puede representarse como una pareja de valores indicando cuantos misioneros y caníbales se van a mover. Por ejemplo el siguiente vector

```python
(0, 1)
```

Indica que se moverán cero misioneros y un canibal. Note que no es necesario representar el bote, pues este siempre se mueve.

Asuma que al principio todos los actores están a la izquierda y al final todos deben estar a la derecha.

In [0]:
class MCProblem(object):
  
    def __init__(self, initial, goal): 
        
        self.initial = initial #por ejemplo (0,3,3,0,0)
        self.goal = goal
        #self.entities = entities
    def actions(self, state):
        #cada state es una 5-tupla
        # state[0] = A el bote esta en el lado izquierdo
        #state[1] = misioneros en el lado izquierdo
        #state[2] = canibales en el lado izquierdo
        #state[3] = misioneros en el lado derecho
        #state[4] = canibales en el lado derecho
        result = []
        if state[0] == "A":
          if state[1] == 0 and state[2] >= 2:
            result.append("Move 2 Cannibal A -> B")
          if state[1] == 0 and state[2] >= 1:
            result.append("Move 1 Cannibal A -> B")
          if state[3] == 0 and state[2] >= 2:
            result.append("Move 2 Cannibal A -> B")
          if state[3] == 0 and state[2] >= 1:
            result.append("Move 1 Cannibal A -> B")
          if state[2] >= 2 and state[1] >= state[2] and state[3] >= state[4]+2:
            result.append("Move 2 Cannibal A -> B")
          if state[1] > 2 and state[1] - 2 >= state[2] and state[3]+2 >= state[4]: 
            result.append("Move 2 Missionary A -> B")
          if state[1] == 2 and state[1] >= state[2]: 
            result.append("Move 2 Missionary A -> B")
          if state[2] >= 1 and state[1] >= state[2] and state[3] >= state[4]+2:
            result.append("Move 1 Cannibal A -> B")
          if state[1] > 1 and state[1] - 1 >= state[2] and state[3]+1 >= state[4]:
            result.append("Move 1 Missionary A -> B")
          if state[1] == 1 and state[1] >= state[2]:
            result.append("Move 1 Missionary A -> B")
          if state[1] >= 1 and state[2] >= 1 and state[1] - 1 >= state[2] -1 and state[3]+1 >= state[4]+1:
            result.append("Move 1 Missionary and 1 Cannibal A -> B")
          
        else:
          if state[3] == 0 and state[4] >= 2:
            result.append("Move 2 Cannibal B -> A")
          if state[3] == 0 and state[4] >= 1:
            result.append("Move 1 Cannibal B -> A")
          if state[1] == 0 and state[4] >= 2:
            result.append("Move 2 Cannibal B -> A")
          if state[1] == 0 and state[4] >= 1:
            result.append("Move 1 Cannibal B -> A")
          if state[4] >= 2 and state[3] >= state[4] and state[1] >= state[2]+2:
            result.append("Move 2 Cannibal B -> A")
          if state[3] > 2 and state[3] - 2 >= state[4] and state[1]+2 >= state[2]: 
            result.append("Move 2 Missionary B -> A")
          if state[3] == 2 and state[3] >= state[4]: 
            result.append("Move 2 Missionary B -> A")
          if state[4] >= 1 and state[3] >= state[4] and state[1] >= state[2]+1:
            result.append("Move 1 Cannibal B -> A")
          if state[3] > 1 and state[3] - 1 >= state[4] and state[1]+1 >= state[2]:
            result.append("Move 1 Missionary B -> A") 
          if state[3] == 1 and state[3] >= state[4]:
            result.append("Move 1 Missionary B -> A")
          if state[3] >= 1 and state[4] >= 1 and state[3] - 1 >= state[4] -1 and state[1]+1 >= state[2]+1:
            result.append("Move 1 Missionary and 1 Cannibal B -> A")
          
        return result
      
    def result(self, state, action): 
        newstate = list(state)
        if action == "Move 2 Cannibal A -> B":
          newstate[2] = state[2]-2
          newstate[4] = state[4]+2
          newstate[0] = "B"
          
        elif action == "Move 2 Missionary A -> B":
          newstate[1] = state[1]-2
          newstate[3] = state[3]+2
          newstate[0] = "B"
          
        elif action == "Move 1 Cannibal A -> B":
          newstate[2] = state[2]-1
          newstate[4] = state[4]+1
          newstate[0] = "B"
          
        elif action == "Move 1 Missionary A -> B":
          newstate[1] = state[1]-1
          newstate[3] = state[3]+1
          newstate[0] = "B"
          
        elif action == "Move 1 Missionary and 1 Cannibal A -> B":
          newstate[1] = state[1]-1
          newstate[2] = state[2]-1
          newstate[3] = state[3]+1
          newstate[4] = state[4]+1
          newstate[0] = "B"
          
        elif action == "Move 2 Cannibal B -> A":
          newstate[2] = state[2]+2
          newstate[4] = state[4]-2
          newstate[0] = "A"
        
        elif action == "Move 2 Missionary B -> A":
          newstate[1] = state[1]+2
          newstate[3] = state[3]-2
          newstate[0] = "A"
        
        elif action == "Move 1 Cannibal B -> A":
          newstate[2] = state[2]+1
          newstate[4] = state[4]-1
          newstate[0] = "A"
        
        elif action == "Move 1 Missionary B -> A":
          newstate[1] = state[1]+1
          newstate[3] = state[3]-1
          newstate[0] = "A"
        
        elif action == "Move 1 Missionary and 1 Cannibal B -> A":
          newstate[1] = state[1]+1
          newstate[2] = state[2]+1
          newstate[3] = state[3]-1
          newstate[4] = state[4]-1
          newstate[0] = "A"
          
        return tuple(newstate)
    def is_goal(self, state):        
        return self.goal == state
    
    def action_cost(self, s, a, s1): 
        return 1
    
    def h(self, node):               
        return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
  
        

### 2.2   Use un método de búsqueda para encontrar una solución óptima al problema

Cree una función que calcule la secuencia de acciones que resuelva el problema así como los estados (incluidos el inicial y el final) que se  visitan al ejecutar la solución. Cuántos movimientos requiere la solución óptima?

In [0]:
def breadth_first_search_MC(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [0]:
# metodo que entrega las acciones usadas para resolver un problema
def path_actions_MC(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]

# metodo que entrega los estados usados para resolver un problema
def path_states_MC(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [12]:
p2_2 = MCProblem(("A",3,3,0,0),("B",0,0,3,3))
soln2_2 = breadth_first_search_MC(p2_2)
path_actions_MC(soln2_2), path_states_MC(soln2_2)

(['Move 2 Cannibal A -> B',
  'Move 1 Cannibal B -> A',
  'Move 2 Cannibal A -> B',
  'Move 1 Cannibal B -> A',
  'Move 2 Missionary A -> B',
  'Move 1 Missionary and 1 Cannibal B -> A',
  'Move 2 Missionary A -> B',
  'Move 1 Cannibal B -> A',
  'Move 2 Cannibal A -> B',
  'Move 1 Cannibal B -> A',
  'Move 2 Cannibal A -> B'],
 [('A', 3, 3, 0, 0),
  ('B', 3, 1, 0, 2),
  ('A', 3, 2, 0, 1),
  ('B', 3, 0, 0, 3),
  ('A', 3, 1, 0, 2),
  ('B', 1, 1, 2, 2),
  ('A', 2, 2, 1, 1),
  ('B', 0, 2, 3, 1),
  ('A', 0, 3, 3, 0),
  ('B', 0, 1, 3, 2),
  ('A', 0, 2, 3, 1),
  ('B', 0, 0, 3, 3)])

###Solución optima

La Solución optima usando el algoritmo de BFS es de 11 acciones

### 2.3  Estados alcanzables desde el estado inicial

Cree una función que calcule todos los estados alcanzables desde el estado inicial, es decir estados para los que existe una secuencia de acciones que lleva del estado inicial a ellos. ¿Son todos los estados posibles alcanzables?

In [13]:
#Clase Nodo que muestra todos los estados
class Node_p:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
        print(self.__dict__)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node_p('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node_p('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.  

{'state': 'failure', 'parent': None, 'action': None, 'path_cost': inf}
{'state': 'cutoff', 'parent': None, 'action': None, 'path_cost': inf}


In [0]:
def breadth_first_search_MC_AS(problem): #Grafo que crea todo el arbol
    "Search shallowest nodes in the search tree first."
    node = Node_p(problem.initial)
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [0]:
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node_p(s1, node, action, cost)

In [16]:
p2_3 = MCProblem(("A",3,3,0,0),("B",0,0,3,3))
soln2_3 = breadth_first_search_MC_AS(p2_3)

{'state': ('A', 3, 3, 0, 0), 'parent': None, 'action': None, 'path_cost': 0}
{'state': ('B', 3, 1, 0, 2), 'parent': <('A', 3, 3, 0, 0)>, 'action': 'Move 2 Cannibal A -> B', 'path_cost': 1}
{'state': ('B', 3, 2, 0, 1), 'parent': <('A', 3, 3, 0, 0)>, 'action': 'Move 1 Cannibal A -> B', 'path_cost': 1}
{'state': ('B', 2, 2, 1, 1), 'parent': <('A', 3, 3, 0, 0)>, 'action': 'Move 1 Missionary and 1 Cannibal A -> B', 'path_cost': 1}
{'state': ('A', 3, 3, 0, 0), 'parent': <('B', 3, 1, 0, 2)>, 'action': 'Move 2 Cannibal B -> A', 'path_cost': 2}
{'state': ('A', 3, 2, 0, 1), 'parent': <('B', 3, 1, 0, 2)>, 'action': 'Move 1 Cannibal B -> A', 'path_cost': 2}
{'state': ('A', 3, 3, 0, 0), 'parent': <('B', 3, 2, 0, 1)>, 'action': 'Move 1 Cannibal B -> A', 'path_cost': 2}
{'state': ('A', 3, 2, 0, 1), 'parent': <('B', 2, 2, 1, 1)>, 'action': 'Move 1 Missionary B -> A', 'path_cost': 2}
{'state': ('A', 3, 3, 0, 0), 'parent': <('B', 2, 2, 1, 1)>, 'action': 'Move 1 Missionary and 1 Cannibal B -> A', 'path_c

![texto alternativo](https://github.com/jsbecerrab/Imagenes-inteligentes/blob/master/diagrama.png?raw=true)

Existen 7 estados posibles por cada lado del rio.  
(A,3,3,0,0),(A,3,2,0,1),(A,2,2,1,1),(A,1,1,2,2),(A,0,3,3,0),(A,3,1,0,2),(A,0,2,3,1)  
(B,0,0,3,3),(B,3,2,0,1),(B,2,2,1,1),(B,1,1,2,2),(B,3,0,0,3),(B,3,1,0,2),(B,0,2,3,1)  
Como se puede observar también el arbol tiene una altura de 14, por lo que cada estado posible aparece por lo menos una vez en cada nivel, haciendo que todos los estados posibles sean alcanzables desde el estado inicial.